In [1]:
import json
from requests import get, post, put

In [2]:
#Test Get Request
def getRequest (url):
    #URL = "http://127.0.0.1:5002/business"
    r = get(url=url)
    dataRetrieved = r.json()
    return dataRetrieved

#Test Post Request
def postRequest (url):
    #url = 'http://127.0.0.1:5002/business'
    retrieve = post(url, json={'data':'remember'}).json()
    #Should retrieve json object
    return retrieve

def putRequst (url, inputJson):
    retrieve = put(url, json=inputJson).json()
    return retrieve

{'You sent': {'feature': 'price', 'positiveOrNegative': 'negative'},
 'Result': {'business_id': '4501',
  'Restaurant name': 'Lai Wah Heen',
  'Cuisine Type': 'Dim Sum',
  'Price': '$$$',
  'Rating': '3.5',
  'Distance': '1.0km'}}

In [24]:
currentRecommendation = getRequest('http://127.0.0.1:5002/business')
print(currentRecommendation)

{'business_id': '4501',
 'name': 'Lai Wah Heen',
 'cuisine': 'Dim Sum',
 'price': '$$$',
 'rating': '3.5',
 'distance': '1.0km',
 'addText': ''}

## Take in critique value, this step mimics the request sent by fulfillment

In [48]:
#Construct the critiquing dictionary 
#categories
#restaurant -> boolean value, anything another than none should be taken
#price
#rating
#distance
#keywords
#critiqueValue
#positiveOrNegative

while True:
    satisfied = 'None'
    feature = 'None'
    positiveOrNegative = 'None'
    critiqueValue = 'None'
    critiqueDict = {}
    
    currentRecommendation = getRequest('http://127.0.0.1:5002/business')
    print('currentRecommendation:', currentRecommendation)
    
    currentCuisineType = currentRecommendation['cuisine']
    #print('Recommending...')
    #displayRestaurantInfo(recommendIndex, current_user_item_predict,df,ID_dictionary)
    
    #Current recommendation cuisine type 
    #currentCuisineType = df[df['business_num_id'] == current_user_item_predict[recommendIndex]]\
    #                                                    .categories.unique()[0].split(', ')
    #currentPriceLabel = IP_dictionary[current_user_item_predict[recommendIndex]]
    #currentRating = IS_dictionary[current_user_item_predict[recommendIndex]]
    #currentDistance = ID_dictionary[current_user_item_predict[recommendIndex]]
    
    #First testing cuisine type
    print('\n????????????????')
    
    while satisfied.lower() not in ['yes', 'no', 'stop']: 
        satisfied = input("You Like? ('yes', 'no') ").strip().lower()
    
    if satisfied == 'stop' or satisfied == 'yes':
        print('BYE :)')
        break
        
    #When satisfied is NO, take in feature
    while feature.lower() not in ['name', 'cuisine', 'price', 'distance', 'rating', 'stop']: 
        feature = input("What feature to critique: (name, cuisine, price, rating, distance)").strip().lower()
    
    if feature == 'stop':
        break
        
    #Add value into dictionary 
    critiqueDict['feature'] = feature
    
    #Take in Positive or nagative 
    if feature != 'name':
        while positiveOrNegative.lower() not in ['positive', 'negative', 'stop']:
            positiveOrNegative = input("Positive or negative: ").strip().lower()
    
    if positiveOrNegative == 'stop':
        break
    
    #Add value into dictionary 
    critiqueDict['positiveOrNegative'] = positiveOrNegative
    
    #Only ask for critique value when not critiquing restaurant name, or not negatively critiquing price 
    if feature != 'name' and not(feature == 'price' and positiveOrNegative == 'negative')\
    and not(feature == 'distance' and positiveOrNegative == 'negative'):
    #and not(feature == 'rating' and positiveOrNegative == 'negative')\
     
        #The valid values to be critiuqed that can pass in 
        validCritiqueValueList = []
        #negatively critique current cuisine type, should only enter current cruisine type
        if 'cuisine' in feature and 'negative' in positiveOrNegative:
            validCritiqueValueList = currentCuisineType.lower().split(',') + ['stop']
            outPutString = '(' + currentCuisineType +')'
        elif 'cuisine' in feature and 'positive' in positiveOrNegative:
            validCritiqueValueList = [cat.strip().lower() for cat in categoryList] 
            categories = list(set(IC_dictionary.keys())) + ['stop']
            outPutString = '(' + categories[0] +categories[1] +categories[2] +'...)'
        #Can only enter cheapter or more expensive
        elif 'price' in feature:
            validCritiqueValueList = ['cheaper', 'more expensive','stop']
            outPutString = '(cheaper, more expensive)'
        elif 'rating' in feature:
            validCritiqueValueList = [str(star/10) for star in range(0,51,1)] + ['stop']
            outPutString = '(0 ~ 5.0 with 0.1 increment)'
        #positively critique distance
        elif 'distance' in feature:
            validCritiqueDis = [str(i/10) for i in range(0,int((maxDistance+0.5)*10),5)][1:] 
            validCritiqueValueList = ['closer', 'further'] + validCritiqueDis + ['stop']
            outPutString = '(closer, further, or distance in range 0.5 ~' + str(maxDistance) + 'in every 0.5km)' 
        #Prompt to ask critique value
        while critiqueValue not in validCritiqueValueList:
            critiqueValue = input("Critique value: " + outPutString).strip().lower()
        
        if critiqueValue == 'stop':
            break
            
        #Add value into dictionary 
        critiqueDict['critiqueValue'] = critiqueValue
            
    print('????????????????\n')
    print('constructed json:', critiqueDict)
    
    stopOrNot = input("Stop?").strip().lower()
    
    if stopOrNot == 'stop' or satisfied == 'yes':
        print('BYE :)')
        break
    #At the end, send out the post request

currentRecommendation: {'business_id': '4501', 'name': 'Lai Wah Heen', 'cuisine': 'Dim Sum', 'price': '$$$', 'rating': '3.5', 'distance': '1.0km', 'addText': ''}

????????????????
You Like? ('yes', 'no') no
What feature to critique: (name, cuisine, price, rating, distance)cuisine
Positive or negative: positive


NameError: name 'categoryList' is not defined

In [29]:
validCritiqueValueList

['Dim Sum', 'stop']

In [47]:
retrieve = put('http://127.0.0.1:5002/business', json=critiqueDict).json()
retrieve

{'message': 'Internal Server Error'}

In [8]:
postRequest('http://127.0.0.1:5002/business')

{'message': 'Internal Server Error'}

In [6]:
#POST request
# URL = "http://127.0.0.1:5002/business"
# data = json.dumps({'ok': 'okok'})
# r = post(url=URL, data = data)
# dataRetrieved = r.text

In [12]:
# url = 'http://127.0.0.1:5002/business'
# myobj = {'somekey': 'somevalue'}
# okk = Request(url, urlencode(myobj).encode())
# x = urlopen(okk).read().decode()

# print(x)

In [40]:
from urllib.parse import urlencode
from urllib.request import Request, urlopen

url = 'https://httpbin.org/post' # Set destination URL here
post_fields = {'foo': 'bar'}     # Set POST fields here

request = Request(url, urlencode(post_fields).encode())
json = urlopen(request).read().decode()
print(json)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "foo": "bar"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "7", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "json": null, 
  "origin": "142.1.71.146, 142.1.71.146", 
  "url": "https://httpbin.org/post"
}



In [1]:
def detect_intent_texts(project_id, session_id, texts, language_code):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    import dialogflow_v2 as dialogflow
    session_client = dialogflow.SessionsClient()

    session = session_client.session_path(project_id, session_id)
    print('Session path: {}\n'.format(session))

    for text in texts:
        text_input = dialogflow.types.TextInput(
            text=text, language_code=language_code)

        query_input = dialogflow.types.QueryInput(text=text_input)

        response = session_client.detect_intent(
            session=session, query_input=query_input)

        print('=' * 20)
        print('Query text: {}'.format(response.query_result.query_text))
        print('Detected intent: {} (confidence: {})\n'.format(
            response.query_result.intent.display_name,
            response.query_result.intent_detection_confidence))
        print('Fulfillment text: {}\n'.format(
            response.query_result.fulfillment_text))

In [ ]:
detect_intent_texts()